In [2]:
# ========================================
# KNESERJEVI GRAFI - tabela α, α_odd, α(G^2)
# ========================================

import itertools
from sage.all import *
from sage.numerical.mip import MixedIntegerLinearProgram, MIPSolverException

# -------------------------------
# 1) Kneserjev graf K(n,k)
# -------------------------------
def kneser_graph(n, k):
    """
    Vrne Sage graf K(n,k) in seznaišč (kot tuple).
    Vozlišča so vse k-podmnožice {1,...,n}.
    Povezava med dvema vozliščema, če sta disjunktna.
    """
    V = [tuple(c) for c in itertools.combinations(range(1, n + 1), k)]
    G = Graph(multiedges=False, loops=False)
    G.add_vertices(V)

    for i in range(len(V)):
        A = set(V[i])
        for j in range(i+1, len(V)):
            B = set(V[j])
            if A.isdisjoint(B):
                G.add_edge(V[i], V[j])
    return G, V

# -------------------------------
# 2) Kvadrat grafa G^2
# -------------------------------
def square_graph(G):
    """
    Vrne kvadrat grafa G^2
    """
    H = Graph()
    H.add_vertices(G.vertices())
    V = G.vertices()
    N = {u: set(G.neighbors(u)) for u in V}

    for i in range(len(V)):
        u = V[i]
        for j in range(i+1, len(V)):
            v = V[j]
            if u != v and (G.has_edge(u,v) or len(N[u].intersection(N[v])) > 0):
                H.add_edge(u, v)
    return H

# -------------------------------
# 3) α(G) z ILP
# -------------------------------
def alpha_ilp(G, solver=None):
    """
    Vrne največjo kardinalnost neodvisnega grafa α(G) z ILP
    """
    p = MixedIntegerLinearProgram(maximization=True, solver=solver)
    x = p.new_variable(binary=True)
    for u,v,_ in G.edges():
        p.add_constraint(x[u] + x[v] <= 1)
    p.set_objective(sum(x[u] for u in G.vertices()))
    p.solve()
    sol_x = p.get_values(x)
    return int(round(sum(sol_x[u] for u in G.vertices())))

# -------------------------------
# 4) α_odd(G) z ILP
# -------------------------------

def alpha_odd_ilp(G, solver=None, timelimit_seconds=None):
    """
    α_odd(G) po ILP:
      max Σ_u x_u
      s.t. x_u ∈ {0,1}, y_u ∈ {0,1}, z_u ∈ Z_{≥0}
           x_u + x_v ≤ 1              ∀(u,v) ∈ E
           Σ_{v∈N(u)} x_v ≤ deg(u)·y_u   ∀u
           y_u + Σ_{v∈N(u)} x_v = 2 z_u  ∀u
    + (neobvezno) linking y_u ≤ 1 - x_u, in zgornja meja za z_u.
    """
    p = MixedIntegerLinearProgram(maximization=True, solver=solver)
    x = p.new_variable(binary=True)    # u ∈ T?
    y = p.new_variable(binary=True)    # u ima soseda v T?
    z = p.new_variable(integer=True)   # parnostni števec

    V = G.vertices()
    N = {u: list(G.neighbors(u)) for u in V}
    deg = {u: len(N[u]) for u in V}

    # cilj
    p.set_objective(sum(x[u] for u in V))

    # neodvisnost
    for (u, v) in G.edges(labels=False):
        p.add_constraint(x[u] + x[v] <= 1)

    # parnostni pogoji
    for u in V:
        # z_u >= 0 (integralno)
        p.add_constraint(z[u] >= 0)

        # (neobvezno, a priporočljivo) linking: če u ∈ T, naj bo y_u = 0
        p.add_constraint(y[u] <= 1 - x[u])

        if deg[u] == 0:
            # Brez sosedov: Σ x_v = 0, zato y_u = 0 in z_u = 0
            p.add_constraint(y[u] == 0)
            p.add_constraint(0 == 2 * z[u])
        else:
            # Big-M na VSOTI sosedov (tesno: M_u = deg(u))
            p.add_constraint(sum(x[v] for v in N[u]) <= deg[u] * y[u])

            # parnost: y_u + Σ x_v je sod (2 z_u)
            p.add_constraint(y[u] + sum(x[v] for v in N[u]) - 2 * z[u] == 0)

            # (neobvezno, a koristno) zgornja meja za z_u
            p.add_constraint(z[u] <= (deg[u] + 1) // 2)

    # Časovna meja solverja (če želiš)
    if timelimit_seconds is not None:
        try:
            p.solver_parameter("timelimit", float(timelimit_seconds))
        except Exception:
            pass

    # reši
    try:
        p.solve()
    except MIPSolverException as e:
        print(f"⚠️ ILP solver ni našel rešitve: {e}")
        return None

    sol_x = p.get_values(x)
    alpha_odd = int(round(sum(sol_x[u] for u in V)))
    return alpha_odd



In [3]:
import os
import csv
import math
from sage.all import *

# -------------------------------
# 1) Nastavitve
# -------------------------------
OUTPUT_DIR = "Kneser_rezultati"
os.makedirs(OUTPUT_DIR, exist_ok=True)

CSV_PATH = os.path.join(OUTPUT_DIR, "kneser_scan.csv")

# glava CSV
header = ["n", "k", "|V|", "alpha", "alpha_odd", "alpha_sq", "eq_alpha_odd_alpha", "eq_alpha_odd_alpha_sq"]
with open(CSV_PATH, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(header)

# -------------------------------
# 2) Obseg za n in k
# -------------------------------
# Tukaj lahko nastaviš max n, da še ne bo predolgo. Za začetek n ≤ 9 je hitro.
n_max = 11

# -------------------------------
# 3) Skener za KG(n,k)
# -------------------------------
for n in range(1, n_max + 1):
    for k in range(1, n//2 + 1):  # k < n/2, ker sicer graf ni zanimiv
        G, V = kneser_graph(n, k)
        # alpha(G)
        alphaG = alpha_ilp(G)
        # alpha_odd(G)
        alpha_od = alpha_odd_ilp(G)
        # alpha(G^2)
        alpha_sq = alpha_ilp(square_graph(G))
        # preverjanje enakosti
        eq1 = alpha_od == alphaG
        eq2 = alpha_od == alpha_sq

        # zapis v CSV
        row = [n, k, len(V), alphaG, alpha_od, alpha_sq, eq1, eq2]
        with open(CSV_PATH, "a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(row)

        # informativni izpis v terminal
        print(f"KG({n},{k}) |V|={len(V)}: alpha={alphaG}, alpha_odd={alpha_od}, alpha(G^2)={alpha_sq} | eq1={eq1}, eq2={eq2}")

print(f"\n✅ Rezultati shranjeni v: {CSV_PATH}")


KG(2,1) |V|=2: alpha=1, alpha_odd=1, alpha(G^2)=1 | eq1=True, eq2=True
KG(3,1) |V|=3: alpha=1, alpha_odd=1, alpha(G^2)=1 | eq1=True, eq2=True
KG(4,1) |V|=4: alpha=1, alpha_odd=1, alpha(G^2)=1 | eq1=True, eq2=True
KG(4,2) |V|=6: alpha=3, alpha_odd=3, alpha(G^2)=3 | eq1=True, eq2=True
KG(5,1) |V|=5: alpha=1, alpha_odd=1, alpha(G^2)=1 | eq1=True, eq2=True
KG(5,2) |V|=10: alpha=4, alpha_odd=3, alpha(G^2)=1 | eq1=False, eq2=False
KG(6,1) |V|=6: alpha=1, alpha_odd=1, alpha(G^2)=1 | eq1=True, eq2=True
KG(6,2) |V|=15: alpha=5, alpha_odd=5, alpha(G^2)=1 | eq1=True, eq2=False
KG(6,3) |V|=20: alpha=10, alpha_odd=10, alpha(G^2)=10 | eq1=True, eq2=True
KG(7,1) |V|=7: alpha=1, alpha_odd=1, alpha(G^2)=1 | eq1=True, eq2=True
KG(7,2) |V|=21: alpha=6, alpha_odd=3, alpha(G^2)=1 | eq1=False, eq2=False
KG(7,3) |V|=35: alpha=15, alpha_odd=15, alpha(G^2)=7 | eq1=True, eq2=False
KG(8,1) |V|=8: alpha=1, alpha_odd=1, alpha(G^2)=1 | eq1=True, eq2=True
KG(8,2) |V|=28: alpha=7, alpha_odd=7, alpha(G^2)=1 | eq1=True

KeyboardInterrupt: 

In [4]:
# -------------------------------
# α(G) z EKR + ILP
# -------------------------------
def alpha_kneser(G, n, k, solver=None):
    """
    Vrne največjo neodvisno množico α(G) za Kneserjev graf K(n,k).
    Če n >= 2k, uporabimo EKR formulo: binomial(n-1, k-1)
    Če n < 2k, uporabimo ILP.
    """
    if n >= 2*k:
        return binomial(n-1, k-1)
    else:
        # fallback ILP
        return alpha_ilp(G, solver=solver)


In [5]:
import os
import csv
import math
from sage.all import *

# -------------------------------
# 1) Nastavitve
# -------------------------------
OUTPUT_DIR = "Kneser_rezultati"
os.makedirs(OUTPUT_DIR, exist_ok=True)

CSV_PATH = os.path.join(OUTPUT_DIR, "kneser_ekr_scan.csv")

# glava CSV
header = ["n", "k", "|V|", "alpha", "alpha_odd", "alpha_sq", "eq_alpha_odd_alpha", "eq_alpha_odd_alpha_sq"]
with open(CSV_PATH, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(header)

# -------------------------------
# 2) Obseg za n in k
# -------------------------------
# Tukaj lahko nastaviš max n, da še ne bo predolgo. Za začetek n ≤ 9 je hitro.
n_max = 9

# -------------------------------
# 3) Skener za KG(n,k)
# -------------------------------
for n in range(1, n_max + 1):
    for k in range(1, n//2 + 1):  # k < n/2, ker sicer graf ni zanimiv
        G, V = kneser_graph(n, k)
        # alpha(G)
        alphaG = alpha_kneser(G, n, k)
        # alpha_odd(G)
        alpha_od = alpha_odd_ilp(G)
        # alpha(G^2)
        alpha_sq = alpha_ilp(square_graph(G))
        # preverjanje enakosti
        eq1 = alpha_od == alphaG
        eq2 = alpha_od == alpha_sq

        # zapis v CSV
        row = [n, k, len(V), alphaG, alpha_od, alpha_sq, eq1, eq2]
        with open(CSV_PATH, "a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(row)

        # informativni izpis v terminal
        print(f"KG({n},{k}) |V|={len(V)}: alpha={alphaG}, alpha_odd={alpha_od}, alpha(G^2)={alpha_sq} | eq1={eq1}, eq2={eq2}")

print(f"\n✅ Rezultati shranjeni v: {CSV_PATH}")


KG(2,1) |V|=2: alpha=1, alpha_odd=1, alpha(G^2)=1 | eq1=True, eq2=True
KG(3,1) |V|=3: alpha=1, alpha_odd=1, alpha(G^2)=1 | eq1=True, eq2=True
KG(4,1) |V|=4: alpha=1, alpha_odd=1, alpha(G^2)=1 | eq1=True, eq2=True
KG(4,2) |V|=6: alpha=3, alpha_odd=3, alpha(G^2)=3 | eq1=True, eq2=True
KG(5,1) |V|=5: alpha=1, alpha_odd=1, alpha(G^2)=1 | eq1=True, eq2=True
KG(5,2) |V|=10: alpha=4, alpha_odd=3, alpha(G^2)=1 | eq1=False, eq2=False
KG(6,1) |V|=6: alpha=1, alpha_odd=1, alpha(G^2)=1 | eq1=True, eq2=True
KG(6,2) |V|=15: alpha=5, alpha_odd=5, alpha(G^2)=1 | eq1=True, eq2=False
KG(6,3) |V|=20: alpha=10, alpha_odd=10, alpha(G^2)=10 | eq1=True, eq2=True
KG(7,1) |V|=7: alpha=1, alpha_odd=1, alpha(G^2)=1 | eq1=True, eq2=True
KG(7,2) |V|=21: alpha=6, alpha_odd=3, alpha(G^2)=1 | eq1=False, eq2=False
KG(7,3) |V|=35: alpha=15, alpha_odd=15, alpha(G^2)=7 | eq1=True, eq2=False
KG(8,1) |V|=8: alpha=1, alpha_odd=1, alpha(G^2)=1 | eq1=True, eq2=True
KG(8,2) |V|=28: alpha=7, alpha_odd=7, alpha(G^2)=1 | eq1=True

KeyboardInterrupt: 